In [ ]:
# Example of how to scale any process over a large area
# GEE tends to run out of memory if complicated exports are run over a large area
# In order to avoid these errors, you may have to break up exports into smaller areas you manage instead of relying on GEE to figure it out for you
####################################################################################################

try:
  import geeViz.getImagesLib as getImagesLib
except:
  !python -m pip install geeViz
  import geeViz.getImagesLib as getImagesLib


#Module imports
# import geeViz.getImagesLib as getImagesLib
import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
ee = getImagesLib.ee

#Set up to mapper objects to use
#Can use the default one first
Map = getImagesLib.Map

print('done')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.4/676.4 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.2 MB/s eta 0:00:00
  Created wheel for simpledbf: filename=simpledbf-0.2.6-py3-none-any.whl size=13785 sha256=af0c651215ac5bc2f3e8e175ece5cedd911586aa5af9c2d630fbf69140e9f455
  Stored in directory: /root/.cache/pip/wheels/e5/41/13/ebdef29165b9309ec4e235dbff19eca8b6759125b0924ad430
Successfully built simpledbf
Initializing GEE
No credentials found
Will attempt ee.Authenticate
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=nC8ybH5ULU5JfURRKwy_2XA-SNMZeW_-naW6UO78Tqo&tc=6TSJKPtKFjE6upvVo

In [ ]:
Map.clearMap()

# First, we'll view the tiles used in the current CONUS LCMS workflow
lcms_CONUS_composites = ee.ImageCollection('projects/lcms-tcc-shared/assets/Composites/Composite-Collection-yesL7-1984-2020')\
                                                .filter(ee.Filter.calendarRange(2022,2022,'year'))

# Pull the geometry of each tile in the composites
lcms_composites_tile_geo = lcms_CONUS_composites.map(lambda f:ee.Feature(f.geometry()).copyProperties(f,['studyAreaName']))

# Add the tiles and a composite for reference
Map.addLayer(lcms_CONUS_composites.mosaic(),getImagesLib.vizParamsTrue10k,'Example CONUS 2022 LCMS Composite')
Map.addLayer(lcms_composites_tile_geo,{},'LCMS Composite Tile Geometry')

Map.centerObject(lcms_composites_tile_geo)
Map.turnOnInspector()
Map.view()

Adding layer: Example CONUS 2022 LCMS Composite
Adding layer: LCMS Composite Tile Geometry
Starting webmap
Using default refresh token for geeView: /root/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /content
Colab Proxy URL: https://c9j3nflx4kp-496ff2e9c6d22116-8001-colab.googleusercontent.com/geeView/?accessToken=None


In [ ]:
Map.clearMap()

# Here is an example of how to create a pyramid of tiles at various scales
# Generally you would start with the biggest tile possible and work your way down till you stop having memory issues
# Currently, LCMS uses 480km tiles (with a 900m buffer) for most processing
lcms_CONUS_studyArea = ee.FeatureCollection('projects/lcms-292214/assets/CONUS-Ancillary-Data/conus')
lcms_CONUS_projection = lcms_CONUS_composites.first().projection()

def getGrid(studyArea,projection,size):
  grid = studyArea.geometry().coveringGrid(projection.atScale(size))
  Map.addLayer(grid,{},'Tile Grid {}m'.format(size))
  return grid

grid480= getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,480000)
getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,240000)
getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,120000)
getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,60000)
Map.addLayer(lcms_CONUS_studyArea,{},'LCMS CONUS Study Area')

Map.view()


Adding layer: Tile Grid 480000m
Adding layer: Tile Grid 240000m
Adding layer: Tile Grid 120000m
Adding layer: Tile Grid 60000m
Adding layer: LCMS CONUS Study Area
Starting webmap
Using default refresh token for geeView: /root/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /content
Colab Proxy URL: https://c9j3nflx4kp-496ff2e9c6d22116-8001-colab.googleusercontent.com/geeView/?accessToken=None


In [ ]:
Map.clearMap()
# As an example of how to use this approach to create composites
ids = grid480.aggregate_histogram('system:index').keys().getInfo()
for id in ids[:2]:
  print(id)
  tile = grid480.filter(ee.Filter.eq('system:index',id)).geometry().buffer(900)

  Map.addLayer(tile,{},'Tile {}'.format(id))

Map.view()

-1,-1
Adding layer: Tile -1,-1
-1,-2
Adding layer: Tile -1,-2
Starting webmap
Using default refresh token for geeView: /root/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /content
Colab Proxy URL: https://c9j3nflx4kp-496ff2e9c6d22116-8001-colab.googleusercontent.com/geeView/?accessToken=None
